# Web Scraping Manchester United Players' Stats

In [1]:
import pandas as pd
import requests
import re
import numpy as np

url = 'https://fbref.com/robots.txt'
response = requests.get(url)
print(response.text)

User-agent:*
# Disallow: /cbb/
# Disallow: /cfb/
# Disallow: /olympics/

# Disallow: /awards/
# Disallow: /blog/
# Disallow: /boxscores/
# Disallow: /coaches/
# Disallow: /draft/
# Disallow: /executives/
# Disallow: /friv/
# Disallow: /hof/
# Disallow: /leaders/
# Disallow: /play-index/
# Disallow: /players/
# Disallow: /route.cgi
# Disallow: /schools/
# Disallow: /search/
# Disallow: /stadiums/
# Disallow: /static/
# Disallow: /teams/
# Disallow: /years/

Disallow: /feedback/
Disallow: /linker/
Disallow: /my/

Disallow: /news/
Disallow: /en/news/
Disallow: /pt/news/
Disallow: /de/news/
Disallow: /fr/news/
Disallow: /es/news/
Disallow: /it/news/
Disallow: /news/

Disallow: /req/
Disallow: /short/
Disallow: /nocdn/


User-agent: AhrefsBot
Disallow: /

User-agent: AhrefsBot/5.0
Disallow: /


## sitemaps generated by copyit/sitemaps/build_sitemaps.pl
## 
Sitemap: https://fbref.com/sitemaps/sitemap.xml



In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
from bs4 import BeautifulSoup
import requests
import time, os

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver"  # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

## 2018-2019 Season Stats

In [5]:
manutd_1819_site = 'https://fbref.com/en/squads/19538871/2018-2019/Manchester-United'

In [6]:
page = requests.get(manutd_1819_site).text
soup = BeautifulSoup(page, 'html5')

### Standard Stats

In [7]:
#standard stats table
player_table = soup.find('table', id='stats_standard_ks_1889')

In [8]:
player_name = [header for header in player_table.find_all('th', class_='left')]
manutd_players = [name.text for th in player_name for name in th.find_all('a')]

In [9]:
player_row = [row for row in player_table.find_all('tr')]

In [10]:
players_stats = {}

for player in player_row[2:]:
    items = player.find_all('td')
    players_stats[player] = [i.text for i in items]

In [11]:
players_stats_list = [stats for stats in players_stats.values()]

In [12]:
manutd_players_stats = {}
manutd_players_stats.update(zip(manutd_players, players_stats_list))

In [13]:
#dataframe of manutd players' stats
manutd_1819 = pd.DataFrame(manutd_players_stats).T
manutd_1819.columns = [
    'Nation', 'Pos', 'Age', 'MP', 'Starts', 'Min', 'Gls', 'Ast', 'PK', 'PKatt',
    'CrdY', 'CrdR', 'P90Gls', 'P90Ast', 'P90G+A', 'P90G-PK', 'P90G+A-PK',
    'ExpectedxG', 'ExpectednpxG', 'ExpectedxA', 'P90xG', 'P90xA', 'P90xG+xA',
    'P90npxG', 'P90npxG+xA', 'Matches'
]

In [14]:
#dropping unneded columns
manutd_1819 = manutd_1819.drop([
    'Pos', 'Nation', 'PK', 'PKatt', 'CrdR', 'P90Gls', 'P90G+A', 'P90G-PK',
    'P90G+A-PK', 'ExpectedxG', 'ExpectednpxG', 'ExpectedxA', 'P90xG', 'P90xA',
    'P90xG+xA', 'P90npxG', 'P90npxG+xA', 'Matches'
],
                               axis=1)

#dropping players who never played a premier league game in the season
manutd_1819 = manutd_1819.drop(['Lee Grant', 'Sergio Romero'])

#data cleaning
manutd_1819 = manutd_1819.replace(r'^\s*$', np.nan, regex=True)
manutd_1819['Min'] = manutd_1819['Min'].str.replace(',', '')
manutd_1819.iloc[:-2, :] = manutd_1819.iloc[:-2, :].astype('float64')

In [15]:
manutd_1819

,Age,MP,Starts,Min,Gls,Ast,CrdY,P90Ast
David de Gea,27,38,38,3420,0,0,1,0
Paul Pogba,25,35,34,3006,13,9,6,0.27
Victor Lindelöf,24,30,29,2601,1,1,1,0.03
Luke Shaw,23,29,29,2591,1,4,11,0.14
Ashley Young,33,30,28,2569,2,2,9,0.07
Nemanja Matić,29,28,28,2436,1,0,7,0
Marcus Rashford,20,33,26,2334,10,6,3,0.23
Chris Smalling,28,24,24,2127,1,0,1,0
Romelu Lukaku,25,32,22,2136,12,0,4,0
Jesse Lingard,25,27,19,1663,4,2,3,0.11


### Shooting Stats

In [16]:
# #shooting stats table
# shooting_table = soup.find('table', id="stats_shooting_ks_1889")

In [17]:
# shooting_table

In [18]:
# shooting_name = [header for header in shooting_table.find_all('th', class_='left')]
# shooting_players = [name.text for th in shooting_name for name in th.find_all('a')]

### Passing Stats

In [19]:
# #passing stats table
# passing_table = soup.find('table', id='stats_passing_ks_1889')

In [20]:
# passing_table

## Exporting 2018-2019 Stats

In [21]:
manutd_1819.to_csv('manutd_1819_stats.csv')